In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import *
import os
import numpy as np
import requests
from bs4 import BeautifulSoup
import sys
import datetime
import io
from PS_modules.PS_DatabaseFunctions import db_read,db_read_valuation_prices, db_read_valuation_FxRates

In [ ]:
portfolio = 'POLAR STAR QUANT'

def makeDump(fund):
    global date
    value_date = pd.Timestamp(date)
    
    positions = db_read('''Select
            PolarStar1.accounts.Portfolio,
            PolarStar1.trades.PS_ID,
            PolarStar1.trades.Strategy,
            PolarStar1.trades.Identifier,
            Sum(PolarStar1.trades.Quantity) As Quantity,
            PolarStar1.identifiers.Strike,
            PolarStar1.identifiers.Description,
            PolarStar1.identifiers.First_Notice_Date,
            PolarStar1.identifiers.Last_Trade_Date,
            PolarStar1.identifiers.Option_Expire_Date,
            PolarStar1.contract_specifications.Contract_Size,
            PolarStar1.contract_specifications.Multiplier,
            PolarStar1.contract_specifications.Currency,
            PolarStar1.identifiers.Market,
            PolarStar1.contract_specifications.Contract_Group,
            PolarStar1.contract_specifications.Exchange,
            PolarStar1.contract_specifications.Contract,
            PolarStar1.contract_specifications.Commodity,
            PolarStar1.contract_specifications.Exposure_Region,
            PolarStar1.trades.Account_Code,
            PolarStar1.accounts.Counterparty_Name,
            PolarStar1.identifiers.Contract_Type,
            PolarStar1.portfolios.Base_Currency,
            PolarStar1.identifiers.Undl_PS_ID_1,
            PolarStar1.identifiers.Undl_PS_ID_2,
            PolarStar1.strategies.Strategy_CommodityGroup,
            PolarStar1.identifiers.Contract_MonthCode,
            PolarStar1.identifiers.Contract_Year
        From
            PolarStar1.trades Left Join
            PolarStar1.identifiers On PolarStar1.trades.PS_ID = PolarStar1.identifiers.PS_ID Left Join
            PolarStar1.contract_specifications On
                    PolarStar1.identifiers.Contract_Code = PolarStar1.contract_specifications.Contract_Code Left Join
            PolarStar1.accounts On PolarStar1.trades.Account_Code = PolarStar1.accounts.Account_Code Left Join
            PolarStar1.portfolios On PolarStar1.accounts.Portfolio = PolarStar1.portfolios.Portfolio_Name Left Join
            PolarStar1.strategies On PolarStar1.trades.Strategy = PolarStar1.strategies.Strategy
        Where
            PolarStar1.trades.Trade_Date <= "{x}" And
            PolarStar1.accounts.Portfolio In ('POLAR STAR LIMITED', 'POLAR STAR SPECTRUM', 'POLAR STAR QUANT')
        Group By
            PolarStar1.accounts.Portfolio,
            PolarStar1.trades.PS_ID,
            PolarStar1.trades.Strategy,
            PolarStar1.trades.Identifier,
            PolarStar1.identifiers.Strike,
            PolarStar1.identifiers.Description,
            PolarStar1.identifiers.First_Notice_Date,
            PolarStar1.identifiers.Last_Trade_Date,
            PolarStar1.identifiers.Option_Expire_Date,
            PolarStar1.contract_specifications.Contract_Size,
            PolarStar1.contract_specifications.Multiplier,
            PolarStar1.contract_specifications.Currency,
            PolarStar1.identifiers.Market,
            PolarStar1.contract_specifications.Contract_Group,
            PolarStar1.contract_specifications.Exchange,
            PolarStar1.contract_specifications.Contract,
            PolarStar1.contract_specifications.Commodity,
            PolarStar1.contract_specifications.Exposure_Region,
            PolarStar1.trades.Account_Code,
            PolarStar1.accounts.Counterparty_Name,
            PolarStar1.identifiers.Contract_Type,
            PolarStar1.portfolios.Base_Currency,
            PolarStar1.identifiers.Undl_PS_ID_1,
            PolarStar1.identifiers.Undl_PS_ID_2,
            PolarStar1.strategies.Strategy_CommodityGroup,
            PolarStar1.identifiers.Contract_MonthCode,
            PolarStar1.identifiers.Contract_Year
        Having
            Sum(PolarStar1.trades.Quantity) != 0'''.format(x=value_date.strftime('%Y%m%d')))
    positions['Date'] = value_date
    positions['FX_PS_ID'] = positions.Currency + positions.Base_Currency+'_FX'
    prices = db_read_valuation_prices(end_date=value_date,select_contracts=positions.PS_ID.unique())
    prices = prices[['Date','PS_ID','Value']]
    prices.columns=['Date','PS_ID','Price']
    fx_rates = db_read_valuation_FxRates(end_date=value_date)
    fx_rates.columns = ['Date','FX_Rate','Data_Source','FX_PS_ID']
    deltas = db_read('''Select
        PolarStar1.valuations_model_option_greeks.Date,
        PolarStar1.valuations_model_option_greeks.PS_ID,
        PolarStar1.valuations_model_option_greeks.Delta
    From
        PolarStar1.valuations_model_option_greeks
    Where
        PolarStar1.valuations_model_option_greeks.Date < '{x}'
    Group By
        PolarStar1.valuations_model_option_greeks.Date,
        PolarStar1.valuations_model_option_greeks.PS_ID,
        PolarStar1.valuations_model_option_greeks.Delta'''.format(x=value_date.strftime('%Y%m%d')))
    deltas = deltas.groupby('PS_ID',as_index=False).tail(1)
    positions1 = pd.merge_asof(positions,prices,on='Date',by='PS_ID')
    positions1 = pd.merge_asof(positions1,prices.rename(columns={'Price':'Underlying_Price','PS_ID':'Undl_PS_ID_1'}),on='Date',by='Undl_PS_ID_1')
    positions1 = pd.merge_asof(positions1,fx_rates.drop(columns='Data_Source'),on='Date',by='FX_PS_ID')
    positions1 = pd.merge_asof(positions1,deltas,on='Date',by='PS_ID')
    positions1.Delta.fillna(1,inplace=True)
    positions1['Delta Position'] = positions1.Quantity * positions1.Delta
    positions1.Underlying_Price.fillna(positions1.Price,inplace=True)
    positions1['Nominal USD'] = positions1['Delta Position'] * positions1.Underlying_Price*positions1.Contract_Size*positions1.Multiplier*positions1.FX_Rate
    positions1['Nominal ZAR'] = positions1['Delta Position'] * positions1.Underlying_Price*positions1.Contract_Size*positions1.Multiplier*(1/positions1.FX_Rate)
    # add CONCA column
    positions1 = positions1.loc[positions.Portfolio == fund]
    positions1['CONCA'] = positions1['Strategy'] + ' ' +  positions1['Contract_Type'] + ' ' + positions1['Strike'].apply(lambda x: str(x) if str(x).lower() != 'nan' else '')
    # df = df.rename(columns={'Account_Code':'Account Code',
    #                         'Quantity':'SumOfNumber',
    #                         'Option_Expire_Date':'Option Expire Date',
    #                        'First_Notice_Date':''})

    # cols = pd.read_csv('cols.txt').columns

    # create new frame and cols for dump and rename appropriately
    cols = ['Portfolio', 'Account Code', 'Strategy', 'Type', 'Strike', 'CONCA',
           'SumOfNumber', 'Commodity', 'First Notice Date', 'Last Trade Date',
           'Option Expire Date', 'Identifier', 'Description', 'Contract Month',
           'Contract', 'Asset Class', 'Commodity Group', 'Delta Position',
           'Nominal USD', 'Nominal ZAR', 'Exchange', 'Geo Region']
    positions1.columns = positions1.columns.str.replace('_',' ')
    positions1 = positions1.rename(columns={'Exposure Region':'Geo Region','Contract Type':'Type',
                              'Strategy CommodityGroup':'Commodity Group',
                              'Quantity':'SumOfNumber','Contract Group':'Asset Class',
                             'Contract MonthCode':'Contract Month'})
    positions1 = positions1[cols]
    
    man_x = {'CURRENCY':'Other cash and cash equivalents (excluding governement securities)',
             'INDEX':'Other Structured/securitised products'}
    fund_x = {'CURRENCY':'Foreign Exchange for Hedging Purposes',
             'INDEX':'Other Structured/securitised products'}
    
    positions1['Manager Category'] = positions1['Asset Class'].apply(lambda x: man_x[x] if x in man_x.keys() else 'Other Commodities/ Agricultural Products')
    positions1['Fund Category'] = positions1['Commodity Group'].apply(lambda x: fund_x[x] if x in fund_x.keys() else 'Other Commodities/ Agricultural Products')

    # Write to excel
    display(positions1)
    positions1.to_excel('Dump for {}'.format(str(value_date.month) + '-' + str(value_date.year)) + ' {}'.format(portfolio) + '.xlsx')
    return print('Finished')

# Initialized variables
def initialvalues(makeJoined,database = False):
    global date
    
    global File1
    global File2
    global Ref_file
    
    global sheetName1
    global sheetName2

    global key_file
    global NAV
    global CashU
    global OTC_Margin
    global keys
    global BankCodes
    global Banks
    global key
    global InvestorGroups
    global numberCodes
    
    Files = [element for element in os.listdir() if 'xlsx' in element.lower() or 'xls' in element.lower()]
    
    if makeJoined == False:
        Op1 = OptionMenu(front,Dump,*Files).grid(row=1,column=0)
        Op2 = OptionMenu(front,Ref,*Files).grid(row=3,column=0)
    else:
        Op1 = OptionMenu(master,Dump,*Files).grid(row=1,column=1)
        Op11 = OptionMenu(master,Dump2,*Files).grid(row=3,column=1)
        File2 = Dump2.get()
        sheetName2 = str(d['e2'].get())
    Op2 = OptionMenu(front,Keys,*Files).grid(row=5,column=0)
    
    if database == True:
        File1 = 'Dump for {}'.format(str(pd.Timestamp(date).month) + '-' + str(pd.Timestamp(date).year)) + ' {}'.format(portfolio) + '.xlsx'
        Ref_file = File1
        key_file = Keys.get()
    else:
        File1 = Dump.get()
        key_file = Keys.get()
        Ref_file = Ref.get()
    
    sheetName1 = str(d['e1'].get())
    date = str(d['e3'].get())
    NAV = float(d['e4'].get().replace(',',''))
    CashU = float(d['e5'].get().replace(',',''))
    OTC_Margin = float(d['e6'].get().replace(',',''))

    # Key dataset
    keys ={'Safex':'XSAF','CME':'XCME','Cbot':'XCBT','Nymex':'NYMX','Nybot':'XNYF','ICE':'IEPA','Liffe':'XLIF'}
    key = pd.DataFrame(list(keys.items()),columns=['Exchange','Code'])
    key.set_index(['Exchange'],inplace=True)
    
    #NumberCodes
    numberCodes = {'CURR':163,'FI':148,'OP':166,'FU':166,'FUND':166,'INDEX':166}
    #Codes to banks
    BankCodes = pd.read_excel('keys.xlsx',sheet_name='BankCodes')
    BankCodes.set_index('Code',inplace=True)
    BankCodes = BankCodes['Bank']
    
    Banks = ['RJO','JPM','Macq','SG','RMB','ABSA']
    
    # InvestorGroups
    InvestorGroups = pd.read_excel('keys.xlsx',sheet_name='InvestGroups')


# Return EUR to Dollar rates
def GrabRate(date):
    global rate
    
    # Building blocks for the URL
    entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/' # Using protocol 'https'
    resource = 'data'           # The resource for data queries is always'data'
    flowRef ='EXR'              # Dataflow describing the data that needs to be returned, exchange rates in this case
    key = 'D.USD.EUR.SP00.A'    # Defining the dimension values, explained below
    
    request_url = entrypoint + resource + '/'+ flowRef + '/' + key
    
    # Define the parameters
    parameters = {'startPeriod': date, 'endPeriod': date}
    
    # Make the HTTP request again, now requesting for CSV format
    response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})

    # Read the response as a file into a Pandas DataFrame
    df = pd.read_csv(io.StringIO(response.text))
    
    rate = float(df['OBS_VALUE'].values[0])
    return print(rate)

def checkMaster():
    dfRef = pd.read_excel(Ref_file,index_col=0)
    Sheet1 = pd.read_excel(File1,sheet_name=sheetName1,index_col=0)
    cols_to_use = ['Identifier'] + dfRef.columns.difference(Sheet1.columns).tolist()
    print(cols_to_use)
#     SheetFinal = pd.read_excel('Master {}'.format(File1.split('.xlsx')[0]) +'.xlsx')
#     main(makeJoined=False,checkMark = True,Sheet = SheetFinal)
    try:
        print("Checking if master file exists ..")
        SheetFinal = pd.read_excel('Master {}'.format(File1.split('.xlsx')[0]) +'.xlsx')
        if SheetFinal[cols_to_use].isnull().values.sum()==0:
            main(makeJoined=False,checkMark = True,Sheet = SheetFinal)
        else:
            print("Your master file still has blank entries.")
    except:
        print("ERROR: Master file doesn't exist - created one")
        display(Sheet1)
        SheetFinal = pd.merge(Sheet1,dfRef[cols_to_use], on=['Identifier'],how='left',suffixes=(' ',' ')).drop_duplicates()
        # Remove spaces at beginning and end of column names
        SheetFinal.columns = SheetFinal.columns.str.strip()
        Writer1 = pd.ExcelWriter('Master {}'.format(File1.split('.xlsx')[0]) +'.xlsx',engine='xlsxwriter')
        SheetFinal.to_excel(Writer1,sheet_name='MasterSheet')
        Workbook1 = Writer1.book
        Worksheet1 = Writer1.sheets['MasterSheet']
        Worksheet1.set_column('A:BC',30)
        Writer1.save()
        
def main(**args):
    GrabRate(date)
    if args['checkMark'] == False:
        if args['makeJoined'] == True:
            Dump1 = pd.read_excel(File1,sheet_name = sheetName1)
            Dump2 = pd.read_excel(File2,sheet_name = sheetName2)
            Suffix = 'Joined'
            Sheet1 = pd.concat([Dump1,Dump2])
        else:
            Sheet1 = pd.read_excel(File1,sheet_name = sheetName1)
            Suffix = ''
    codes = pd.read_excel(key_file,sheet_name='Entities')

    codes.set_index(['Entity','Name'],inplace=True)
    try:
        Sheet1 = args['Sheet']
        Suffix = ''
    except:
        pass
    Ex1 = Sheet1.groupby(['Exchange','Commodity']).sum().round(1)
    Mg1 = Sheet1.groupby(['Manager Category','Commodity']).sum().round(1)
    Fd1 = Sheet1.groupby(['Contract']).sum().round(1)
    Fd2 = Sheet1.groupby(['Contract','Commodity','Exchange','Manager Category','Description','Last Trade Date'],as_index=False).sum().round(1)
    Fd2['Last Trade Date'] = Fd2['Last Trade Date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').strftime('%m/%d/%Y'))
    Fd3 = Sheet1.groupby(['Geo Region','Commodity']).sum().round(1)
    Fd4 = Sheet1.groupby(['Fund Category','Commodity']).sum().round(1)
    Fd6 = Sheet1.groupby(['Description','Commodity']).sum().round(1)
    Fd7 = Sheet1.groupby(['Account Code','Commodity']).sum().round(1)
    Fd8 = Sheet1.groupby(['Commodity']).sum().round(1)
    Fd12 = Sheet1.groupby(['Manager Category','Commodity']).sum().reset_index()


    Fd12 = Fd12[['Manager Category','Commodity','Nominal USD']]
    Fd12['ABS'] = Fd12['Nominal USD'].apply(lambda x: abs(x))
    Fd12['LONG'] = Fd12['Nominal USD'].apply(lambda x: None if x < 0 else x)
    Fd12['SHORT'] = Fd12['Nominal USD'].apply(lambda x: None if x > 0 else x)
    Fd12.set_index(['Manager Category','Commodity'],inplace=True)

    Indices = pd.DataFrame(0,columns=['Nominal USD','ABS','LONG','SHORT'],index=['Other Commodities/ Agricultural Products','Other Commodities/ Livestock',
               'Other Structured/securitised products','Other cash and cash equivalents (excluding governement securities)'
              ,'Other commodities/Industrial metals','Other Commodities/ Other','Precious metals/Gold','Precious Metals/Other'
              ,'Energy/ Crude Oil','Energy/Natural gas'])

    Indices.index.name='Manager Category'
    Indices = Indices.reset_index()

    Fd12Sum = Fd12.groupby(['Manager Category']).sum()
    Fd12Sum = Fd12Sum.reset_index()
    
    Fd12Desc = Sheet1.groupby(['Description']).sum().round(1)/2
    Fd12Desc = Fd12Desc[['Nominal USD']]
    Fd12Desc.reset_index(inplace=True)
    Fd12Desc['Index'] = Fd12Desc['Description'].apply(lambda x: numberCodes[x] if x in numberCodes else '?')
    Fd12Desc.set_index('Index',inplace=True)
    
    Fd12Final = pd.merge(Indices,Fd12Sum,on='Manager Category',how='outer',suffixes=('_',''))
    Fd12Final.drop([x for x in Fd12Final if x.endswith('_')],axis=1,inplace=True)

    Fd12Final.set_index(['Manager Category'],inplace=True)
    Fd12Sum.set_index(['Manager Category'],inplace=True)


    #Some variables for INDEXP
    try:
        TbillL = Fd12.xs('TBILL',level=1, drop_level=False)['LONG'].sum()
        TbillS = abs(Fd12.xs('TBILL',level=1, drop_level=False)['SHORT'].sum())
    except:
        TbillL = '-'
        TbillS = '-'
    try:
        MACQL = Fd12.xs('MACQ_RPI',level=1, drop_level=False)['LONG'].sum()
        MACQS = abs(Fd12.xs('MACQ_RPI',level=1, drop_level=False)['SHORT'].sum())
    except:
        print('No MACQ_RPI commodity found')
        MACQL ='-'
        MACQS = '-'
    try:
        ok = Fd12Sum.loc['Other cash and cash equivalents (excluding governement securities)']['ABS'] - TbillL
    except:
        print('Other cash and ash equivalents (excluding governement securities) not found')
        ok = '-'
    try:
        E129L = Fd12Sum.loc['Other Commodities/ Livestock']['LONG']
        E129S = abs(Fd12Sum.loc['Other Commodities/ Livestock']['SHORT'])
    except:
        E129L = '-'
        E129S = '-'
        print('Other commodities/livestock index not found')
    try:
        E130L = Fd12Sum.loc['Other Commodities/ Agricultural Products']['LONG']
        E130S = abs(Fd12Sum.loc['Other Commodities/ Agricultural Products']['SHORT'])
    except:
        E130L = '-'
        E130S = '-'

    MASTER = pd.DataFrame(data = [[TbillL,TbillS],[MACQL,MACQS],[ok,None],[E129L,E129S],[E130L,E130S]],
                          columns = ['LONG','SHORT'],index=[77,107,120,129,130])

    #END 
    Fd11 = Sheet1.loc[:, ['Geo Region','Nominal USD']]
    Fd11H = Sheet1.loc[:, ['CONCA','Geo Region','Nominal USD']]
    Fd11H['CONCA'] = Fd11H['CONCA'].apply(lambda x: x if 'hedge' in x.lower() else None)
    Fd11H = Fd11H.dropna(axis=0)
    Fd11H['LONG (USD)'] = Fd11H['Nominal USD'].apply(lambda x: None if x < 0 else x)
    Fd11H['SHORT (USD)'] = Fd11H['Nominal USD'].apply(lambda x: None if x > 0 else x)
    Fd11H.set_index(['Geo Region','CONCA'],inplace=True)
    Fd11H = Fd11H.sort_values(by='Geo Region')

    Fd11HF = Fd11H.groupby(['Geo Region']).sum()
    Fd11['LONG (USD)'] = Fd11['Nominal USD'].apply(lambda x: None if x < 0 else x)
    Fd11['SHORT (USD)'] = Fd11['Nominal USD'].apply(lambda x: None if x > 0 else x)
    Fd11 = Fd11.groupby(['Geo Region']).sum().round(1)

    Fd11 = Fd11.drop(['Nominal USD'],axis=1)
    Fd11H = Fd11H.drop(['Nominal USD'],axis=1)
    Fd11HF = Fd11HF.drop(['Nominal USD'],axis=1)

    Fd10 = pd.DataFrame(0,index=Banks,columns=['Value','%'])

    Fd9 = Sheet1[['Identifier']]
    Fd9 = Fd9.Identifier.unique().tolist()
    Fd9 = pd.DataFrame(Fd9,columns=['Identifier'])
    Fd9 = Fd9.sort_values(by=['Identifier'],ascending=True).reset_index(drop=True)


    Fd8 = Fd8[['Nominal USD']]
    Fd8['ABS'] = Fd8['Nominal USD'].apply(lambda x: abs(x))
    Fd8.drop('TBILL',inplace=True)
    Fd8Sum = Fd8['ABS'].sum()

    Fd7 = Fd7[['Nominal USD']]
    Fd7['ABS'] = Fd7['Nominal USD'].apply(lambda x: abs(x))
    Fd7Sum = Fd7.groupby(['Account Code']).sum().round(1).reset_index()
    Fd7Sum['Bank'] = Fd7Sum['Account Code'].apply(lambda x: BankCodes[x])
    Fd7Final = Fd7Sum.groupby(['Bank']).sum().round(1).sort_values(by=['ABS'],ascending=False)
    Fd7Final['Cash at Bank'] = 0
    Fd7Final['%'] = 0
    Fd7Final['Difference'] = 0

    Fd7Sum.set_index(['Bank','Account Code'],inplace=True)

    # Conc princip

    Fd4 = Fd4[['Nominal USD']]
    Fd4['ABS'] = Fd4['Nominal USD'].apply(lambda x: abs(x))
    Fd4['LONGS'] = Fd4['Nominal USD'].apply(lambda x: None if x < 0 else x)
    Fd4['% LONGS'] = Fd4['LONGS']/Fd4['ABS'].sum()
    Fd4['SHORTS'] = Fd4['Nominal USD'].apply(lambda x: None if x > 0 else x)
    Fd4['% SHORTS'] = abs(Fd4['SHORTS']/Fd4['ABS'].sum())

    #RHS UP data set
    Fd4Sum = Fd4.reset_index()
    Fd4Sum = Fd4Sum.groupby(['Fund Category']).sum().sort_values(by=['ABS'],ascending=False)


    # Reindex for RHS down
    Fd4Final = Fd4Sum.reset_index()

    #RHS down dataset
    Fd4Final = Fd4Final.melt(id_vars=["Fund Category","Nominal USD","ABS","LONGS",'SHORTS'], 
            var_name="Type", 
            value_name="%").sort_values(by='%',ascending=False)
    Fd4Final['Type'] = Fd4Final['Type'].apply(lambda x: x.split('%')[1])

    Fd4Final.set_index(['Fund Category'],inplace=True)
    Fd4Final = Fd4Final[['%','Type']]
    Fd4Final['NET USD'] = Fd4Final['%'] * Fd4['ABS'].sum()

    # END 

    Fd3 = Fd3[['Nominal USD']]
    Fd3['ABS'] = Fd3['Nominal USD'].apply(lambda x: abs(x))
    Fd3Sum = Fd3.groupby(['Geo Region']).sum().round(1)
    Fd3Sum['% Aggregate Asset Value'] = Fd3Sum['ABS']/Fd3Sum['ABS'].sum()
    Fd3Sum['Net'] = Fd3Sum['Nominal USD']
    Fd3Sum['% Portion of NAV'] = Fd3Sum['Net']/NAV
    Fd3Sum['% of NAV to 100%'] = Fd3Sum['% Portion of NAV']/Fd3Sum['% Portion of NAV'].sum()

    Fd2['Type'] = Fd2['Nominal USD'].apply(lambda x: 'SHORT' if x < 0 else 'LONG')
    Fd2['ABS'] = Fd2['Nominal USD'].apply(lambda x: abs(x))

    Fd2 = Fd2[['Contract','Commodity','Exchange','Manager Category','Description','Last Trade Date','ABS','Type']]
    Fd2 = Fd2.sort_values(by=['ABS'],ascending=False)
    Fd2.set_index(['Contract'],inplace=True)

    Fd1 = Fd1[['Nominal USD']]
    Fd1['ABS'] = Fd1['Nominal USD'].apply(lambda x: abs(x))

    #Borrow R GrossExp sheet:
    Fd6 = Fd6[['Nominal USD']]
    Fd6Original = Sheet1.groupby(['Description','Commodity']).sum().round(1)[['Nominal USD']]
    Fd6Original['ABS'] = Fd6Original['Nominal USD'].apply(lambda x: abs(x))
    Fd6['ABS'] = Fd6['Nominal USD'].apply(lambda x: abs(x))
    Fd6Sum = Fd6.groupby(['Description']).sum().round(1)
    Fd6Com = Fd6.reset_index(drop=False)
    OTCList = [element for element in Fd6.reset_index(level=1)['Commodity'] if 'OTC' in element]
    OTC = Fd6Com[Fd6Com['Commodity'].isin(OTCList)]
    
    for element in OTCList:
        Fd6.drop(element,level=1,inplace=True)
        
    Fd6.drop('TBILL',level=1,inplace=True)
    
    try:
        Fd6.drop('CURR',level=0,inplace=True)
    except:
        print('WARNING: CURR Commodity not found')
    
    Fd6FinalSum = Fd6.groupby(['Description'])
    DervGross = Fd6['ABS'].sum()
    Margin = NAV*CashU - OTC_Margin
    Difference = DervGross - Margin
    Percentage = Difference*100/NAV
    OTC = OTC['ABS'].sum()
    Difference_2 = OTC - OTC_Margin
    Percentage_2 = Difference_2*100/NAV
    Gross2 = (DervGross + OTC)*100/NAV

    ETD = pd.DataFrame([DervGross,Margin,Difference,NAV,Percentage,OTC,OTC_Margin,Difference_2,Percentage_2,Gross2],columns = ['Value']
                      ,index = ['DervGross','Margin','Difference',
                                'NAV','Percentage','OTC','OTC Margin','Difference','Percentage','Gross'])

    Mg1Sum = Mg1[['Nominal USD']]
    Mg1Sum = Mg1Sum.rename(columns = {'Nominal USD':'Total'})
    Mg1Sum['ABS'] = Mg1Sum['Total'].apply(lambda x: abs(x))
    Mg1FinalSum = Mg1Sum.groupby(['Manager Category']).sum().round(1)
    Mg1FinalSum = Mg1FinalSum.rename(columns = {'ABS':'NET USD'})
    Mg1FinalSum['NET EUR'] = Mg1FinalSum['NET USD']*(1/rate)


    FinalSum = Ex1[['Nominal USD']]
    try:
        Ex1.drop('MACQ_RPI',level=1,inplace=True)
    except:
        print("WARNING: MACQ_RPI Commodity not found!")
    try:
        Ex1.drop('TBILL',level=1,inplace=True)
    except:
        print("WARNING: TBILL Commoditty not found!")
    for element in OTCList:
        Ex1.drop(element,level=1,inplace=True)

    #OTC's
    Ex2 = Sheet1.groupby(['Commodity'],as_index=False).sum().round(1)
    Ex2 = Ex2[Ex2['Commodity'].str.contains('OTC')]
    Ex2['Exchange'] = 'OTC'
    Ex2.set_index(['Exchange','Commodity'],inplace=True)

    Ex = pd.concat([Ex1,Ex2],ignore_index=False)

    Final = Ex[['Nominal USD']]

    FinalPlus = Final[Ex.groupby('Nominal USD')['Nominal USD'].transform(lambda x: x > 0)].reset_index()
    FinalMinus = Final[Ex.groupby('Nominal USD')['Nominal USD'].transform(lambda x: x < 0)].reset_index()

    Final3 = pd.merge(FinalPlus,FinalMinus, on=['Exchange','Commodity'],how='outer',suffixes = (' +',' -'))
    Final3 = Final3.fillna(0)
    Final4 = Final3.groupby(['Exchange']).sum().round(1)
    Final4['Nominal USD -'] = Final4['Nominal USD -'].apply(lambda x: abs(x))

    Tot = Final4['Nominal USD +'].sum() + Final4['Nominal USD -'].sum()

    Final4['+%'] = Final4['Nominal USD +']*100/(Tot)
    Final4['-%'] = Final4['Nominal USD -']*100/(Tot)

    Final4['NET USD'] = Final4['Nominal USD +'] + Final4['Nominal USD -']

    # Conmost NB
    FdNB = Final4.reset_index()
    FdNB = FdNB.rename(columns = {'Nominal USD +':'LONGS','Nominal USD -': 'SHORTS','+%':'LONG','-%':'SHORT'})


    FdNB['Max USD'] = FdNB[["LONGS", "SHORTS"]].max(axis=1)

    FdNB = FdNB.melt(id_vars=["Exchange","LONGS","SHORTS","NET USD",'Max USD'], 
            var_name="Type", 
            value_name="Sum").sort_values('Sum',ascending=False)

    FdNB['Sum'] = FdNB['Sum']/100

    FdNB.set_index(['Exchange'],inplace=True)


    #END

    Final4['NET EUR'] = Final4['NET USD'] * (1/rate)

    FinalSum = FinalSum.rename(columns = {'Nominal USD':'Total'})
    FinalSum['ABS'] = FinalSum['Total'].apply(lambda x: abs(x))

    
    
    '''
    Opera risk calculations
    '''
    
    operaDf = pd.read_excel('keys.xlsx',sheet_name='OperaRisk')
    operaDf['Sum'] = operaDf['Realised'] + operaDf['Unrealised'] + operaDf['Interest']
    operaDf['Gross'] = operaDf['Sum']/operaDf['AUM']
    operaDf['Sum post-expenses'] = operaDf['Sum'] + operaDf['Other Expenses']
    operaDf['Net'] = operaDf['Sum post-expenses']/operaDf['AUM']
    operaDf['Change in AUM'] = operaDf.AUM.diff()
    operaDf.set_index('Date',inplace=True)

    # To excel
    Writer = pd.ExcelWriter('{}'.format(Suffix) + 'Split'+'.xlsx',engine='xlsxwriter')

    FinalSum.to_excel(Writer,sheet_name = 'Mg Rep - Ex BDown')
    Final4.to_excel(Writer,sheet_name = 'Mg Rep - Ex BDown',startcol= FinalSum.shape[1] + 3)

    Mg1Sum.to_excel(Writer,sheet_name = 'Mg Rep - Instr')
    Mg1FinalSum.to_excel(Writer,sheet_name = 'Mg Rep - Instr',startcol= Mg1Sum.shape[1] + 3)

    codes.to_excel(Writer,sheet_name = 'Fund Rep - Prime Broker')

    Fd1.to_excel(Writer,sheet_name = 'FundRep- Concentra',startrow = 10)
    Fd2.to_excel(Writer,sheet_name = 'FundRep- Concentra',startrow = 10,startcol= Fd1.shape[1] + 2)
    key.to_excel(Writer,sheet_name = 'FundRep- Concentra')

    Fd3.to_excel(Writer,sheet_name = 'FundRep-ConcenGeog')
    Fd3Sum.to_excel(Writer,sheet_name = 'FundRep-ConcenGeog',startcol= Fd3.shape[1] + 3)

    Fd4.to_excel(Writer,sheet_name = 'FundRep-ConcPrincipExp')
    Fd4Sum.to_excel(Writer,sheet_name = 'FundRep-ConcPrincipExp',startrow= Fd4.shape[0] + 3,startcol = 1)
    Fd4Final.to_excel(Writer,sheet_name = 'FundRep-ConcPrincipExp',startcol = Fd4.shape[1] + 3)

    FdNB.to_excel(Writer,sheet_name = 'FundRep-ConMost NB',startrow = 10)
    key.to_excel(Writer,sheet_name = 'FundRep-ConMost NB')

    Fd12.to_excel(Writer,sheet_name='FundRep-IndExp')
    Fd12Final.to_excel(Writer,sheet_name='FundRep-IndExp',startrow=Fd12.shape[0]+3,startcol=1)
    MASTER.to_excel(Writer,sheet_name='FundRep-IndExp',startcol=Fd12.shape[1]+3)
    Fd12Desc.to_excel(Writer,sheet_name='FundRep-IndExp',startcol=Fd12.shape[1]+3,startrow=MASTER.shape[0]+2)

    Fd11.to_excel(Writer,sheet_name = 'Fund Rep - Currency Exp')
#     Fd11H.to_excel(Writer,sheet_name = 'Fund Rep - Currency Exp',startcol=Fd11.shape[1]+3)

    Fd11HF.to_excel(Writer,sheet_name = 'Fund Rep - Currency Exp',startrow = Fd11H.shape[0]+3,startcol=Fd11.shape[1]+3)

    Fd10.to_excel(Writer, sheet_name='Fund Rep - Counter Risk',startrow=2)
    codes.to_excel(Writer, sheet_name='Fund Rep - Counter Risk',startcol=Fd10.shape[1] + 3)
    
    InvestorGroups.to_excel(Writer, sheet_name='FundRep – InvestorGroups')
    Fd6Original.to_excel(Writer,sheet_name = 'FundRep-BorrowR GrossExp')
    Fd6Sum.to_excel(Writer,sheet_name = 'FundRep-BorrowR GrossExp',startcol= Fd6.shape[1] + 3)
    ETD.to_excel(Writer,sheet_name = 'FundRep-BorrowR GrossExp',startcol= Fd6Sum.shape[1] + Fd6.shape[1] + 6 )


    Fd8Final = pd.DataFrame(data = [Fd8Sum.round(2),NAV,Gross2],columns=['Values'],index=['Commitment Exposure of AIF',
                                                                                          'NAV','Gross'])
    writeFrame = pd.DataFrame(data=[['Dump as it is and absolute everything and summed up'],
                                    ['Netting each commodity & summing absolutes'],
                                    ['Commitment can equal gross in this case.']],columns=['Info'],index=['Gross','Com','NOTICE:'])
    
    Fd8Final.to_excel(Writer,sheet_name = 'FundRep-BorrowR GrossExp',startrow=Fd6Sum.shape[0]+3,startcol= Fd6.shape[1] + 3)
    writeFrame.to_excel(Writer,sheet_name = 'FundRep-BorrowR GrossExp',startrow=ETD.shape[0]+3,
                        startcol= Fd6Sum.shape[1] + Fd6.shape[1] + 6)
    
    Fd9.to_excel(Writer, sheet_name='FundRep-Op R#ofPosi')

    Fd7.to_excel(Writer,sheet_name = 'FundRep-BorrowSource')
    Fd7Sum.to_excel(Writer,sheet_name = 'FundRep-BorrowSource',startcol= Fd7.shape[1] + 3)
    Fd7Final.to_excel(Writer,sheet_name = 'FundRep-BorrowSource',startrow = Fd7Sum.shape[0]+3,startcol= Fd7.shape[1] + 3)
    
    codes.to_excel(Writer, sheet_name='FundRep-BorrowSource', startrow = 
                   Fd7Sum.shape[0] + Fd7Final.shape[0] + 6, startcol = Fd7.shape[1] + 3)
    
    operaDf.to_excel(Writer,sheet_name = 'Fund Rep - Opera Risk')


    #Formatting
    Workbook = Writer.book
    Worksheet = Writer.sheets['Mg Rep - Ex BDown']
    Worksheet2 = Writer.sheets['Mg Rep - Instr']
    Worksheet3 = Writer.sheets['FundRep- Concentra']
    Worksheet4 = Writer.sheets['FundRep-ConcenGeog']
    Worksheet5 = Writer.sheets['FundRep-ConcPrincipExp']
    Worksheet6 = Writer.sheets['FundRep-ConMost NB']
    Worksheet7 = Writer.sheets['Fund Rep - Currency Exp']
    Worksheet8 = Writer.sheets['FundRep-BorrowR GrossExp']
    Worksheet9 = Writer.sheets['FundRep-BorrowSource']
    Worksheet11 = Writer.sheets['FundRep-Op R#ofPosi']
    Worksheet12 = Writer.sheets['Fund Rep - Counter Risk']
    Worksheet13 = Writer.sheets['FundRep-IndExp']
    Worksheet14 = Writer.sheets['FundRep – InvestorGroups']
    Worksheet15 = Writer.sheets['Fund Rep - Opera Risk']
    
    # Write codes
    Worksheet8.write('J6','229')
    Worksheet8.write('J10','230')
    Worksheet8.write('J11','233/4')

    codes = Writer.sheets['Fund Rep - Prime Broker']

    Format = Workbook.add_format({'num_format':'###0.00'})
    Format2 = Workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#122057',
        'border': 1, 'font_color':'white','num_format':'###0.00'})
    Format3 = Workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#ff0000',
        'border': 1, 'font_color':'white','num_format':'###0.00'})
    Format3.set_center_across()

    PercForm = Workbook.add_format({'num_format':'0.00%'})
    PercForm2 = Workbook.add_format({'bold':True, 'num_format':'0.00%'})
    PercForm2.set_center_across()
    Dollars = Workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#122057',
        'border': 1, 'font_color':'white','num_format':'###0.00'})
    
    centerForm = Workbook.add_format({'bold':True,'num_format':'###0.00'})
    centerForm.set_center_across()

    Worksheet.set_column('A:L',20,Format)
    Worksheet.set_tab_color('#00B050')

    Worksheet2.set_column('B:I',20,Format)
    Worksheet2.set_column('A:A',60,Format)
    Worksheet2.set_column('F:F',60,Format)
    Worksheet2.set_tab_color('#00B050')

    Worksheet3.set_column('A:L',20,Format)
    Worksheet3.set_column('H:H',60,Format)
    Worksheet3.set_tab_color('#00B050')
    Worksheet3.set_zoom(85)

    Worksheet4.set_column('A:L',20,Format)
    Worksheet4.set_column('I:I',30,PercForm)
    Worksheet4.set_column('K:L',30,PercForm)
    Worksheet4.set_zoom(85)
    Worksheet4.set_tab_color('#00B050')

    Worksheet5.set_column('A:B',40,Format)
    Worksheet5.set_column('C:H',20,Format)
    Worksheet5.set_column('J:J',40,Format)
    Worksheet5.set_column('L:M',20,Format)
    Worksheet5.set_column('F:F',20,PercForm)
    Worksheet5.set_column('H:H',20,PercForm)
    Worksheet5.set_column('K:K',15,PercForm)
    Worksheet5.set_zoom(85)
    Worksheet5.set_tab_color('#00B050')

    Worksheet6.set_column('A:L',20,Format)
    Worksheet6.set_column('G:G',20,PercForm)
    Worksheet6.set_tab_color('#00B050')

    Worksheet7.set_column('A:I',20,Format)
    Worksheet7.set_column('G:G',35,Format)
    Worksheet7.set_tab_color('#00B050')

    Worksheet8.set_column('A:I',20,Format)
    Worksheet8.set_column('F:F',25,Format)
    Worksheet8.set_column('K:M',20,Format)
    Worksheet8.set_tab_color('#00B050')
    Worksheet8.write(5,Fd6Sum.shape[1] + Fd3.shape[1] + 7,str(Percentage.round(2)) + '%',PercForm2)
    Worksheet8.write(9,Fd6Sum.shape[1] + Fd3.shape[1] + 7,str(Percentage_2.round(2)) + '%',PercForm2)
    Worksheet8.write(10,Fd6Sum.shape[1] + Fd3.shape[1] + 7,str(Gross2.round(2)) + '%',PercForm2)

    Worksheet9.set_column('A:D',20,Format)
    Worksheet9.set_column('F:H',35,Format)
    Worksheet9.set_column('I:K',20, Format)

    Worksheet9.set_tab_color('#00B050')

    Worksheet11.set_column('A:H',20,Format)
    Worksheet11.set_tab_color('#00B050')

    Worksheet12.set_column('A:C',20,Format)
    Worksheet12.set_tab_color('#00B050')
    Worksheet12.set_column('F:I',35,Format)

    Worksheet13.set_column('A:B',60,Format)
    Worksheet13.set_column('C:F',20,Format)
    Worksheet13.set_column('H:Q',30,Format)
    Worksheet13.set_zoom(85)
    Worksheet13.set_tab_color('#ff0000')

    
    codes.set_column('A:L',35,Format)
    codes.set_tab_color('#00B050')
    
    Worksheet14.set_column('A:L',35,Format)
    Worksheet14.set_tab_color('#00B050')
    
    Worksheet15.set_column('A:K',20,Format)
    Worksheet15.set_column('H:H',20,PercForm)
    Worksheet15.set_column('J:J',20,PercForm)
    Worksheet15.set_tab_color('#00B050')
    
    Worksheet.write(Final4.shape[0] + 3,FinalSum.shape[1] + Final4.shape[1] + 3,'1:' + str(rate) + ' or ' + str(np.round(1/rate,4)) + ':1',Format2)
    Worksheet.write(Final4.shape[0] + 2,FinalSum.shape[1] + Final4.shape[1] + 3,'Current EUR to USD:',Format2)

    Worksheet2.write(Mg1FinalSum.shape[0] + 3,Mg1Sum.shape[1] + Mg1FinalSum.shape[1] + 3,'1:' + str(rate) + ' or ' + str(np.round(1/rate,4)) + ':1',Format2)
    Worksheet2.write(Mg1FinalSum.shape[0] + 2,Mg1Sum.shape[1] + Mg1FinalSum.shape[1] + 3,'Current EUR to USD:',Format2)

    Worksheet3.write(0,0,'Exchange',Format3)
    Worksheet3.write(0,1,'Code',Format3)

    Worksheet6.write(0,0,'Exchange',Format3)
    Worksheet6.write(0,1,'Code',Format3)

    Worksheet8.write(0,Fd6Sum.shape[1] + Fd6.shape[1] + 6 ,'Cash U: {}'.format(str(CashU*100) + '%'),Format2)

    Worksheet9.write_comment(Fd7Sum.shape[0]+3,Fd7.shape[1] + 6, 'PLEASE FILL IN CASH AT BANK BELOW', {'x_scale': 1.2, 'y_scale': 0.8})
    Worksheet12.write(0,0,'NAV:',Format2)
    Worksheet12.write(0,1,NAV,Dollars)
    Worksheet12.write(0,2,'PLEASE FILL IN % COL',Dollars)

    for i in range(1,Fd10.shape[0]+1):
        Worksheet12.write(2+i,1,'=C{}/100*$B$1'.format(3+i))
    for i in range(1,Fd7Final.shape[0]+1):
        Worksheet9.write(Fd7Sum.shape[0]+3+i,Fd7.shape[1] + 8, '=H{}-I{}'.format(Fd7Sum.shape[0]+4+i,Fd7Sum.shape[0]+4+i)) 
        Worksheet9.write(Fd7Sum.shape[0]+3+i,Fd7.shape[1] + 7, '=H{}*100/I{} & "%"'.format(Fd7Sum.shape[0]+4+i,Fd7Sum.shape[0]+4+i)) 
    Writer.save()
    
if __name__ == '__main__':
    d = {}
    # Tkinter instance for front GUI
    front = Tk()
    # Size of GUI
    front.minsize(width=300, height=80)
    # Title of GUI
    front.title('FCA reporting')
    # Allow to not be resizable
    front.resizable(0,0)
    
    master = tk.Toplevel(front)
    master.title('Join Dumps')
    master.withdraw()
    
    Labels = ["Dump File:","Sheet Name 1:","Reference File","Keys File:","Sheet Name 2:","Date","NAV","CashU","OTC_Margin"]
    masterLabels = ["Dump File 1","Dump File 2"]
    Label(master,text=masterLabels[0]).grid(row=0,column=1)
    Label(master,text=masterLabels[1]).grid(row=2,column=1)
    
    Label(front, text = Labels[0]).grid(row = 0, column = 0)
    Label(front, text = Labels[1]).grid(row = 0, column = 2)
    Label(front, text = Labels[2]).grid(row = 2, column = 0)
    Label(front, text = Labels[3]).grid(row = 4, column = 0)
    Label(front, text = Labels[4]).grid(row = 2, column = 2)
    Label(front, text = Labels[5]).grid(row = 4, column = 2)
    Label(front, text = Labels[6]).grid(row = 6, column = 2)
    Label(front, text = Labels[7]).grid(row = 8, column = 2)
    Label(front, text = Labels[8]).grid(row = 10, column = 2)
    
    for i in range (1,7):
        d["e{}".format(i)] = Entry(front,width=20)
    
    d["e1"].grid(row = 1, column = 2)
    d["e2"].grid(row = 3, column = 2)
    d["e3"].grid(row = 5, column = 2)
    d["e4"].grid(row = 7, column = 2)
    d["e5"].grid(row = 9, column = 2)
    d["e6"].grid(row = 11, column = 2)
    
    Dump = StringVar(front)
    Dump2 = StringVar(front)
    Ref = StringVar(front)
    Keys = StringVar(front)
    
    var1 = IntVar()
    # Save function that writes to results.txt file the present inputs
    def save():
        f = open('entries.txt','w')
        for element in d:
            print(d[element].get())
            f.write(d[element].get() + '\n')
        print("SUCCESS: Results have been saved in results.txt file")
        f.close() 

    # Load function that deletes entries that you put in and inserts the ones from results.txt file
    def load(d,state):
        f = open('entries.txt','r')
        prev = [line.strip('\n') for line in f]
        counter = 0 
        for element in d:
            d[element].delete(0,END)
            d[element].insert(0,prev[counter])
            # If unticked then state is disabled else enabled
            if var1.get() == 0:
                d[element].config(state=state)
            else:
                d[element].config(state='normal')
            counter+=1
        f.close()
    # Will always try and load into the entries the results file when you open the program or else it will return error
    try:
        load(d,'disabled')
    except:
        print('ERROR: results.txt File cannot be found. Please paste it into the current directory!')
    
    
    b1 = Button(front,text='Make Singular',command=lambda:(initialvalues(False),checkMaster()),bg='green').grid(row=13,column=1)
    b2 = Button(front,text='Load Previous',command=lambda:(load(d,'disabled')),bg='orange').grid(row=13,column=2)
    b3 = Button(front,text="Save inputs", command=lambda: save(),bg = 'DeepSkyBlue3').grid(row=14,column = 2)
    b4 = Checkbutton(front,text='Edit Values?',variable=var1,command=lambda: load(d,'disabled')).grid(row=12,column=2)
    b5 = Button(front, text='Quit', command=lambda: (front.destroy()),bg ='IndianRed4').grid(row=13, column=0, sticky=W, pady=4)
    
    #Join buttons
    b6 = Button(front,text='Join',command=lambda:(master.deiconify(),initialvalues(True)),bg='Purple').grid(row=14,column=1)
    b7 = Button(master,text='Make Joined',command=lambda:(initialvalues(True),main(True,False)),bg='green').grid(row=4,column=2)
    b8 = Button(master,text='Hide', command=lambda: (master.withdraw()),bg ='IndianRed4').grid(row=4, column=0, sticky=W, pady=4)
    b9 = Button(front,text='Create from DB', command=lambda: (makeDump(portfolio),initialvalues(False,database = True),checkMaster()),bg ='Orange').grid(row=14, column=0, sticky=W, pady=4)
    initialvalues(False)
    mainloop()

In [ ]:
# from datetime import datetime
print(datetime.now())

In [ ]:
df = pd.DataFrame(columns=['a','b','c','d'])
df.set_index(['a','b'],inplace=True)
display(df)
print(df.shape[0])
df.to_excel('test.xlsx')

In [ ]:
df = pd.DataFrame(data = [['ye',2,3,4],['ye',6,7,8],['no',21,43,34]],columns=['Identifier','b','c','d'])
display(df)
df3 = pd.DataFrame(data = [['ye'],['ye']],columns=['Identifier'])
df2 = pd.merge(df,df3[['Identifier']], on=['Identifier'],how='left',suffixes=(' ',' '))

display(df2.drop_duplicates())